# Capstone Part 2c: Markov Chains Model

We now explore the predictive capabilities of a simple Markov Chains Model.

Abstract: Markov Chains models a finite set of states with probabilities of going from one state to a next. For example, a model could have 2 possible follow-ups to the word "a" in "I have a": 0.5 chance of "cat" and 0.5 chance of "dog". A downside to Markov Chains is that it does not have "memory" of any sort. For a 1st order Markov Chains model, whenever the model runs into the word "a" it will still consider the next word to be either "cat" or "dog" regardless of its context.

Compared to RNN and transformers, Markov Chains is an old method of text generation, considered almost obsolete at this point. However, there are upsides to this method as well which will be discussed in the results section.

Reference: https://techeffigytutorials.blogspot.com/2015/01/markov-chains-explained.html

In [12]:
import time
import numpy as np

### Additional Preprocessing

This is a character based Markov Chains model which works quite similarly to the LSTM model, except it works on simple probabilities it trains on.

In [3]:
with open('dataset/enron6_clean.txt', 'r') as f:
    corpus = f.read()

In [4]:
corpus = corpus.replace(
    '\n',' '
).replace(
    '?','.'
).replace(
    '!','.'
).replace(
    '“','.'
).replace(
    '”','.'
).replace(
    '/',' '
).replace(
    '‘',' '
).replace(
    '-',' '
).replace(
    '’',' '
).replace(
    '\'',' '
).replace(
    '=', ' '
).replace(
    '\\', ' '
).replace(
    '_', ' '
)

In [36]:
len(corpus)

17219655

In [37]:
corpus[0:1000]

'RTO   Grid South, SE Trans, SPP and Entergy Southeast RTO orders are out and have followed through with what we expected from the discussion at the FERC meeting. SPP and Entergy RTO proposals have been rejected because they fail to satisfy the scope and configuration requirements of Order No. . Commission notes that the required discussions between SPP and Entergy and its neighboring RTO TOs has led to no increase in the original scope and configuration. filings by SPP and Entergy were brief, indicating only a lack of interest by other RTOs or utilities in joining to enlarge scope; theyfailed to specify any details of the talks, what changes could be made or what could be fixed to accomodate combination with other RTOs. order states that the Commission favors the development of large, regional transmission organizations reflecting natural markets. Commission indicates that they favor four RTOs   NE, SE, MW and West. Therefore the order requires the participants in SPP and Entergy to p

### Functions Used

In [5]:
# create stochastic matrix function
def create_matrix(corpus, k=4): # k is chain order, default 4
    T = {} # empty matrix
    
    for i in range(len(corpus)-k):
        X = corpus[i:i+k] # slice k characters
        Y = corpus[i+k] # the character after X
        
        if T.get(X) is None: # if X does not exist in matrix yet
            T[X] = {} # create X key
            T[X][Y] = 1 # create 1 instance of Y after X
        else: 
            if T[X].get(Y) is None: # otherwise if Y value does not exist for X key
                T[X][Y] = 1 # create 1 instance of Y after X
            else: # otherwise...
                T[X][Y] += 1 # add 1 instance of Y after X
    
    return T

In [6]:
# convert frequency from stoc matrix to probabilities
def freq2prob(T):     
    for kx in T.keys():
        s = float(sum(T[kx].values())) # sum of total frequencies
        for k in T[kx].keys():
            T[kx][k] = T[kx][k]/s # probability of frequency
                
    return T

In [7]:
# run model
def model(corpus, k=4):
    T = create_matrix(corpus, k)
    T = freq2prob(T)
    return T

In [8]:
def sample_next(char, model, k): 
    char = char[-k:]
    if model.get(char) is None: # if char not found in matrix
        return " "
    
    possible_chars = list(model[char].keys()) # retrieve key from stoch matrix
    possible_values = list(model[char].values()) # retrieve value from stoch matrix
 
    return np.random.choice(possible_chars,p=possible_values)

In [9]:
def generate_text(sentence, k=4, max_len=5):    
    char = sentence[-k:]    
    for ix in range(max_len):
        next_prediction = sample_next(char, model, k)
        sentence += next_prediction
        char = sentence[-k:]
    return sentence

## Modelling

In [10]:
t0 = time.time()
model = model(corpus)
t1 = time.time()

total = t1-t0

In [46]:
total

17.6802077293396

In [49]:
sample_next('Sout', model, 4)

'h'

## Results

In [72]:
# create a new magic function to get samples with text in ipynb cell
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def sample(line):
    print(generate_text(line, k=4, max_len=100))

### Seeded Text Generation Test

In [18]:
%sample
'Let me know'

                PORT IDEAS FOR THE FINAL SECUTED. THE FINAL LOAD ID: ECT@ECT Submission metals Tradi


'Let me know'

In [21]:
%sample
'Regarding'

    Preferred by commended will contained With what is monumer said I want, sing the Independed to d


'Regarding'

Applying it to text generation gives us a few words that still make sense (Let me know port ideas for the final...), but it quickly devolves into nonsense. This is to be expected from a model which operates purely on probabilistic behavior.

### Topic-based Text Generation Test

In [73]:
%sample
'Financial'

      Thanks, MN US to PG&E s Open t make it the addition. Budget in June that coller. Besses the $$


'Financial'

In [74]:
%sample
'Meeting'

        Want telex : Wicketing the need to us this may very informal clearning impresentations from 


'Meeting'

Unsurprisingly, it does not perform well because there is no attention in this model.

### Word Autocompletion Test

In [57]:
print(generate_text('Sincer', k=4, max_len=5))

Sincerely, 


In [60]:
print(generate_text('Regar', k=4, max_len=5))

Regards, m


In [17]:
print(generate_text('Regar', k=4, max_len=5))

Regards ha


However, singular word autocompletion appears to be working rather well! Although several tries have to be performed in order to get the required output, it is still able to produce a word that has appeared in the corpus with just a few characters.

### Conclusions

Advantages of Markov Chains:

1. It is rather easy to implement and trains extremely quickly (17.6802077293396 seconds).

2. It is able to perform singular word autocompletion.

3. It does not require any external libraries.


Disadvantages of Markov Chains:

1. It is rather fickle in the sense that it doesn't predict the required result every time. A custom temperature function could be implemented in order to control this but it kind of defeats the purpose of using a probabilistic model.

2. It cannot generate coherent sentences well

3. It is extremely subsceptible to noise. A perfect set of data is required to use it well.